In [1]:
from bs4 import BeautifulSoup
import requests, json, os
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import pprint
import gensim
import PyPDF2
import os

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.52",
}

params = {
    "q": "",
    "hl": "en",
    "start": 0
}
from tkinter import *
window=Tk()

directory_path = '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb'


In [2]:
#SPARK RELATED CODE
import findspark

findspark.init()

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession \
    .builder \
    .appName('Sparkler') \
    .getOrCreate()

spark.conf.set('spark.sql.caseSensitive', False)

22/11/24 11:18:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/24 11:18:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
def gensimImplementation(documents):
    q = params['q']
    for document in documents:
        print(gensim.utils.simple_preprocess(document))

In [4]:
#Similar Articles
def getSimilarArticles(df, vectorizer, link): 
    print('-------------------getsimilarArticles----------------------------')
    q = [params['q']]
    q_vector = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    print(q_vector)
    sim = {}
    
    for i in range(df.shape[1]):
        sim[i] = np.dot(df.loc[:, i].values, q_vector) / np.linalg.norm(df.loc[:,i]) * np.linalg.norm(q_vector)
        
    sim_sorted = sorted(sim.items(), key = lambda x:x[1], reverse = True)
    print(sim_sorted)
    for k,v in sim_sorted[:5]:
        if v != 0.0:
            print("Similarities: {}".format(v))
            print('Link to the article:', link[k])

In [5]:
def vectorizerMethod(documents_clean):
    # Instantiate a TfIdfVectorizer Object and transform the data to vectors
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documents_clean)
    X = X.T.toarray()

    df = pd.DataFrame(X, index=vectorizer.get_feature_names_out())
    
    print('--------------------vectorizermethod-----------------------')
    print(df.head(2))
    return(df, vectorizer)
    
    #getSimilarArticles(df, vectorizer)

In [6]:
def cleanData(documents):
    documents_clean = []
    for d in documents:
        # Remove Unicode
        document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
        # Remove Mentions
        document_test = re.sub(r'@\w+', '', document_test)
        # Lowercase the document
        document_test = document_test.lower()
        # Remove punctuations
        document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
        # Lowercase the numbers
        document_test = re.sub(r'[0-9]', '', document_test)
        # Remove the doubled space
        document_test = re.sub(r'\s{2,}', ' ', document_test)
        documents_clean.append(document_test)
    print('-------------------cleandata-------------------')
    print(documents_clean[:2])
    return(documents_clean)
    #vectorizerMethod(documents_clean)
#     gensimImplementation(documents=documents_clean)


In [7]:
def collectDocumentFromLinks(link):
    #Retrieve paragrahs from each link, combine each paragrah as a string and save it to docs
    documents = []
    for i in link:
        r = requests.get(i, headers=headers)
        soup = BeautifulSoup(r.content,'html.parser')

        sen = []
        # for springer abstracts
        if soup.find('div', {'id':'Abs1-content'}):
            for i in soup.find('div', {'id':'Abs1-content'}).find_all('p'):
                sen.append(i.text)

        documents.append(' '.join(sen))
    print('Fetched data from each link...')
    print('------------collectDocumentfromlinks-------------------')
    print(documents[:2])
    return(documents)
    #cleanData(documents)

In [8]:
def collectSpringerLinks(soup):
    #Retrieve all popular new links
    link = []
    print(len(soup))
    i=0
    for i in range(0, len(soup)):
        data = soup[i].find_all("div", {"class": "gs_ri"})
        for j in range(len(data)):    

            temp = data[j].find('a')

            if 'link.springer.com/article' in temp['href'] and 'books.google.com' not in temp['href']:
                link.append(temp['href'])
    #             print(temp['href'], j)

    #print(len(link))
    print('---------------collectSpringerlinks-------------------')
    print(link[:2])
    if len(link)>0:
        #collectDocumentFromLinks(link)
        return(link)
    else: 
        print('No links found. Try with other keywords.')

In [9]:
def collectGoogleScholarPages(query):
    params['q'] = query
    soup = []
    while True:
        url = 'https://scholar.google.com/scholar'
        req = requests.get(url, headers=headers, params=params)
        print(req.url)
        tempData = BeautifulSoup(req.content,'html.parser')
#         print(tempData)
        soup.append(tempData)
        if tempData.find('span', {'class': 'gs_ico gs_ico_nav_next'}) and params['start']<=100:
            params['start']+=10
        else:
            break
            
    print('-----------collectGoogleScholarPages-------------')
    #print(soup)
    # call collectSpringerLinks()
    return soup
    #collectSpringerLinks(soup)

In [10]:
###Read PDFs(Papers) from Document Database
def readdocumentdb(directory_path):
    DE = []
    for filename in os.listdir(directory_path):
        if filename[-4:] == '.pdf':
            filepath = directory_path+'/'+filename
            pdfFileObj = open(filepath, 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
            numofpages = pdfReader.numPages
            pageObj = pdfReader.getPage(0)
            text = pageObj.extractText()

            try:
                abstractindex = re.search(r'Abstract', text, re.IGNORECASE)
                introindex = re.search(r'(I. )?(1. )?I*ntroduction', text, re.IGNORECASE)
                abstractend = 0
                introstart = 0

                if abstractindex:
                    abstractend = abstractindex.end()
                if introindex:
                    introstart = introindex.start()

                if introstart == 0:
                    paperAbstract = text[abstractend:]
                else:
                    paperAbstract = text[abstractend:introstart]
            except:
                paperAbstract = text

            paperAbstract = re.sub(r"^\W+", "", paperAbstract.strip())
            pdfFileObj.close()
            A = [filepath, paperAbstract]
            DE.append(A)

    DocumentsExtract = pd.DataFrame(DE)
    DocumentsExtract.columns = ['DocLink', 'Abstract']
    #print(DocumentsExtract)
    return(DocumentsExtract)

In [11]:
# lbl=Label(window, text="Enter keywords to search documents", fg='black', font=("Helvetica", 12))
# lbl.place(relx=.5, rely=.25,anchor= CENTER)

# txtfld=Entry(window, text="Enter keywords (',' separated)", bd=2)
# txtfld.place(relx=.5, rely=.35,anchor= CENTER)

# btn=Button(window, text="Search Documents", fg='black', command=lambda: collectGoogleScholarPages(txtfld.get()))
# btn.place(relx=.5, rely=.45,anchor= CENTER)

# window.title('Sparkler for Documents')
# window.geometry("500x300+250+250")
# window.mainloop()

keywords = input("Enter keywords on which you want to search documents:")

"""
##Reading google scholar
soup = collectGoogleScholarPages(keywords)
links = collectSpringerLinks(soup)
docs = collectDocumentFromLinks(links)
cdocs = cleanData(docs)
features, vector = vectorizerMethod(cdocs)
getSimilarArticles(features, vector, links)



dbdocs = readdocumentdb(directory_path)
dbdocsabstract = dbdocs["Abstract"].to_numpy().tolist()
dbdocslinks = dbdocs["DocLink"].to_numpy().tolist()
cleanabstractdata = cleanData(dbdocsabstract)
featuredocs, vectordocs = vectorizerMethod(cleanabstractdata)
getSimilarArticles(featuredocs, vectordocs, dbdocslinks)
"""

##Reading google scholar
soup = collectGoogleScholarPages(keywords)
links = collectSpringerLinks(soup)
docs = collectDocumentFromLinks(links)

##read all the documents from db
dbdocs = readdocumentdb(directory_path)
dbdocsabstract = dbdocs["Abstract"].to_numpy().tolist()
dbdocslinks = dbdocs["DocLink"].to_numpy().tolist()

combinedlinks = links + dbdocslinks
combineddocs = docs + dbdocsabstract

cleandataset = cleanData(combineddocs)
featureds, vectords = vectorizerMethod(cleandataset)
getSimilarArticles(featureds, vectords, combinedlinks)


Enter keywords on which you want to search documents:Lung Oncology
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=0
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=10
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=20
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=30
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=40
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=50
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=60
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=70
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=80
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=90
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=100
https://scholar.google.com/scholar?q=Lung+Oncology&hl=en&start=110
-----------collectGoogleScholarPages-------------
12
---------------collectSpringerlinks-------------------
['https://link.springer.com/arti

Xref table not zero-indexed. ID numbers for objects will be corrected.


Fetched data from each link...
------------collectDocumentfromlinks-------------------
['Progress in genetic engineering has made it possible to elucidate the molecular biological abnormalities in lung cancer. Mutations in KRAS and P53 genes, loss of specific alleles, and DNA methylation of the tumor suppressor genes were the major abnormalities investigated between 1980 and the 2000s. In 2004, mutations in the epidermal growth factor receptor (EGFR) gene that cause oncogene addiction were discovered in non-small-cell lung cancers (NSCLCs), especially in adenocarcinomas. Because they are strongly associated with sensitivity to EGFR-tyrosine kinase inhibitors (EGFR-TKIs), a great deal of knowledge has been acquired in regard to both EGFR and other genes in the EGFR family and their downstream genes. Moreover, in 2007 the existence of the echinoderm microtubule-associated protein-like 4 (EML4)-anaplastic lymphoma kinase (ALK) fusion gene was discovered in NSCLC; and the same as EGFR-TKIs

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


-------------------cleandata-------------------
['progress in genetic engineering has made it possible to elucidate the molecular biological abnormalities in lung cancer mutations in kras and p genes loss of specific alleles and dna methylation of the tumor suppressor genes were the major abnormalities investigated between and the s in mutations in the epidermal growth factor receptor egfr gene that cause oncogene addiction were discovered in non small cell lung cancers nsclcs especially in adenocarcinomas because they are strongly associated with sensitivity to egfr tyrosine kinase inhibitors egfr tkis a great deal of knowledge has been acquired in regard to both egfr and other genes in the egfr family and their downstream genes moreover in the existence of the echinoderm microtubule associated protein like eml anaplastic lymphoma kinase alk fusion gene was discovered in nsclc and the same as egfr tkis alk inhibitors are being found to be highly effective in lung cancers that have thi